### Метод опорных весторов
Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

In [1]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold #Кросс-валидация
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [8]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

После выполнения этого кода массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе sklearn.feature_extraction.text.TfidfVectorizer. Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

Реализация SVM-классификатора находится в классе sklearn.svm.SVC. Веса каждого признака у обученного классификатора хранятся в поле coef_. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом get_feature_names() у TfidfVectorizer:

Подбор параметров удобно делать с помощью класса sklearn.grid_search.GridSearchCV (При использовании библиотеки scikit-learn версии 18.0.1 sklearn.model_selection.GridSearchCV). Пример использования:

In [23]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)

gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
#gs.fit(X, y)
display(grid)

{'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
        1.e+03, 1.e+04, 1.e+05])}

Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

In [32]:
#gs.cv_results_:
# a.mean_validation_score — оценка качества по кросс-валидации
# a.parameters — значения параметров

### Программа реализации

In [15]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

#Вычисление TF-IDF-признаков для всех текстов
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target

#Cетка параметров для перебора 10^-5 ... 10^5 
grid = {'C': np.power(10.0, np.arange(-5, 6))}

#Кросс-валидация по 5 блокам
cv = KFold(n_splits=5, shuffle=True, random_state=241)

#Обучение и подбор параметра С
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv) #Подбор параметров (В нашем случае C)
gs.fit(X,y)
gs.cv_results_

{'mean_fit_time': array([3.00211968, 3.16744857, 3.09562478, 3.10160151, 2.67761135,
        1.62484107, 1.55921345, 1.60480347, 1.58026052, 1.54081168,
        1.67076912]),
 'std_fit_time': array([0.17778136, 0.15359163, 0.04947239, 0.13319091, 0.14761782,
        0.09809762, 0.01724028, 0.06254661, 0.04491307, 0.02851937,
        0.08421242]),
 'mean_score_time': array([0.73806014, 0.74318805, 0.71383133, 0.7403996 , 0.64839854,
        0.36917791, 0.35781803, 0.36940122, 0.35859804, 0.35279961,
        0.37680974]),
 'std_score_time': array([0.03776966, 0.03147801, 0.01074824, 0.02806663, 0.02684614,
        0.02283174, 0.00752187, 0.02189658, 0.01326751, 0.00886235,
        0.02135754]),
 'param_C': masked_array(data=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                    1000.0, 10000.0, 100000.0],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False],
        fill_value='?',
             dtype=object)

In [16]:
#ВЫВОД Параметр С=1 дает лучший результат на 5 выборках

clf = SVC(kernel='linear', random_state=241) #С=1.0 по умолчанию
clf.fit(X,y)

#10 слов с наибольшим абсолютным значением веса
#Находим слова с наиболее большим весом из get_feature_names()
df = pd.DataFrame(np.transpose(abs(clf.coef_.toarray())), #Берем по модулю потому что 2 класса todense() переводит матрицу из разряженной в обычную 
                   index=np.asarray(vectorizer.get_feature_names()), 
                   columns=["col"])

df_sort = df.sort_values(by='col')[::-1]
display(df_sort[:10].sort_index())

,col
atheism,1.254690
atheists,1.249180
bible,1.130612
god,1.920379
keith,1.097094
moon,1.201611
religion,1.139081
sci,1.029307
sky,1.180132
space,2.663165


In [6]:
df1 = pd.DataFrame(np.transpose(clf.coef_.toarray()), #Берем по модулю потому что 2 класса todense() переводит матрицу из разряженной в обычную 
                   index=np.asarray(vectorizer.get_feature_names()), 
                   columns=["col"])
display(df1.sort_values(by='col'))

,col
god,-1.920379
atheism,-1.254690
atheists,-1.249180
religion,-1.139081
bible,-1.130612
...,...
nasa,1.024223
sci,1.029307
sky,1.180132
moon,1.201611


###### Отрицательные значения соответствиют класс 0, что в свою очередь тема Атеизма, а положительные значения классу 1, что значит тема Космоса

In [17]:
X_test = ["Last fall, Earth was gripped by the news that two large satellites were going to make an uncontrolled fall from orbit: NASA's UARS and Germany's ROSAT. Fortunately, despite many pieces of the satellites surviving to impact Earth's surface no one was hurt and no damage was done. However, if the falls had taken place just a few hours either way of when they did, populated areas could have been in the fall zone. "]
X_test = vectorizer.transform(X_test)
clf.predict(X_test)

array([1], dtype=int32)